# Calculate edge circadian scores of drug-disease pairs in Ruben et al paper 

In [1]:
import numpy as np
import pandas as pd
import src.calculate_edge_circa_score as cal
import requests

In [2]:
# read in processed CircaDB data 
circa_db = pd.read_csv('data/circa_db_mapped.tsv', sep = '\t')
circa_db.head()

,gene_id,Fat SQ_fdr,Fat Visceral_fdr,Aorta_fdr,Artery Coronary_fdr,Artery Tibial_fdr,Colon_fdr,Esophagus_fdr,Heart Atrial_fdr,Liver_fdr,...,Artery Coronary_exp,Artery Tibial_exp,Colon_exp,Esophagus_exp,Heart Atrial_exp,Liver_exp,Lung_exp,Nerve Tibial_exp,Pituitary_exp,Thyroid_exp
0,653635,0.6432,0.1526,0.8443,0.7712,0.9549,0.5059,0.2928,0.6953,0.9732,...,12.300,11.590,12.7200,12.3033,5.369,5.406,13.680,19.480,15.840,19.2550
1,79854,0.7652,0.2412,0.0473,0.0002,0.6314,0.8602,0.0820,0.6481,0.6405,...,5.940,8.419,5.4407,4.6607,2.962,3.240,7.024,12.110,9.898,9.7615
2,643837,0.9075,0.0774,0.7887,0.2173,0.6017,0.3294,0.0696,0.2758,0.3438,...,9.039,11.520,7.1738,7.6350,14.350,3.674,6.016,5.872,18.600,6.5845
3,26155,0.8656,0.4570,0.5405,0.6391,0.8885,0.3902,0.6801,0.8984,0.3532,...,52.240,62.410,50.8450,62.9550,30.670,28.420,57.320,69.660,57.560,66.5800
4,339451,0.8808,0.6235,0.4360,0.4999,0.7446,0.7780,0.6238,0.6645,0.9089,...,13.510,10.800,11.6200,14.5633,4.870,5.979,15.600,9.224,21.420,15.4450


In [3]:
# read in metapath data
metapath_df = pd.read_csv('data/rephetio_significant_metapaths.tsv', header = 0, sep = '\t')
metapath_df.head()

,abbreviation,length,delta_auroc,negative_log_p
0,CtDaGaD,3,12.70%,7.2485
1,CbGbCtD,3,21.70%,6.4962
2,CbGaD,2,14.50%,6.2137
3,CrCbGaD,3,8.22%,6.0926
4,CbGeAlD,3,8.43%,5.1873


In [4]:
metapaths = list(metapath_df.abbreviation)

### Pre-process circadian treatments extracted from Ruben et al

In [5]:
# Ruben et al treatment data 
treatment = pd.read_csv('data/HumCircMed2018v2_mapped.tsv', sep = '\t')

# Filter drug~disease pairs
treatment = treatment[~treatment['drug.trtmnt_drugbank_id'].isna()]
treatment = treatment[~treatment['therapeutic.area_doid'].isna()]

filter_row_id = []
filter_col_id = ['drug.trtmnt','drug.trtmnt_drugbank_id','therapeutic.area',
                 'therapeutic.area_doid','halflife.hrs','effect']
for i in range(0,len(treatment)):
    drug = treatment['drug.trtmnt_drugbank_id'].iloc[i,]
    disease = treatment['therapeutic.area_doid'].iloc[i,]
    effect = str(treatment['effect'].iloc[i,])
    if ('more eff' in effect) or ('none' in effect):
        # whether treatment contains only one drug
        if len(drug.split(',')) == 1: 
            filter_row_id.append(i)
treatment = treatment[filter_col_id].iloc[filter_row_id,:]
treatment.head()

,drug.trtmnt,drug.trtmnt_drugbank_id,therapeutic.area,therapeutic.area_doid,halflife.hrs,effect
24,prednisone,DB00635,asthma,DOID:2841,2.5,more eff
29,theophylline,DB00277,asthma,DOID:2841,8,more eff
30,epinephrine,DB00668,asthma,DOID:2841,0.03,more eff
31,orciprenaline,DB00816,asthma,DOID:2841,6,more eff
33,salmeterol,DB00938,asthma,DOID:2841,5.5,none


### Calculate edge circadian scores of drug-disease pairs

In [6]:
# obtain names of tissues in CircaDB
circa_cols = list(circa_db.columns)
tissues = []
for i in range(0, len(circa_cols)):
    cols_s = circa_cols[i].split('_amp')
    if len(cols_s) > 1:
        tissues.append(cols_s[0])
tissues = list(np.unique(tissues))

In [7]:
# calculate edge circadian scores 
rb_tissues_scores = []
rb_tissues_notes = []
rb_tissues_metapaths = []
rb_tissues_paths = []
# iterate drug~disease pair
for tr in range(0, len(treatment)):
    print(tr,'\n')
    drug = treatment['drug.trtmnt_drugbank_id'].iloc[tr,]
    disease = treatment['therapeutic.area_doid'].iloc[tr,]
    score, note, n_meta, n_path = cal.calculate_edge_circa_score(drug, disease, tissues, circa_db, 
                                                                 query_metapath = metapaths)
    rb_tissues_scores.append(score)
    rb_tissues_notes.append(note)
    rb_tissues_metapaths.append(n_meta)
    rb_tissues_paths.append(n_path)

0 

1 

2 

3 

4 

5 

6 

7 

8 

9 

10 

11 

12 

13 

14 

15 

16 

17 

18 

19 

20 

21 

22 

23 

24 

25 

26 

27 

28 

29 

30 

31 

32 

33 

34 

35 

36 

37 

38 

39 

40 

41 

42 

43 

44 

45 

46 

47 

48 

49 

50 

51 



In [10]:
# score dataframe
rb_df = pd.DataFrame(rb_tissues_scores)
rb_df.columns = tissues
# note dataframe
note_df = pd.DataFrame(rb_tissues_notes)
note_df.columns = ['score_note']
# metapath count dataframe 
meta_count_df = pd.DataFrame(rb_tissues_metapaths)
meta_count_df.columns = ['metapath_cout']
# path count dataframe 
path_count_df = pd.DataFrame(rb_tissues_paths)
path_count_df.columns = ['path_cout']
# combine dataframes and output 
combine_df = pd.concat([treatment.reset_index(drop = True), rb_df, note_df, meta_count_df, path_count_df], axis=1)
combine_df.to_csv('data/HumCircMed2018v2_mapped_edge_circa_scores.tsv', sep = '\t', na_rep = 'NA', 
                  float_format = '%.4f', index = False)
combine_df.head()

,drug.trtmnt,drug.trtmnt_drugbank_id,therapeutic.area,therapeutic.area_doid,halflife.hrs,effect,Aorta,Artery Coronary,Artery Tibial,Colon,...,Fat Visceral,Heart Atrial,Liver,Lung,Nerve Tibial,Pituitary,Thyroid,score_note,metapath_cout,path_cout
0,prednisone,DB00635,asthma,DOID:2841,2.5,more eff,0.034652,0.157328,0.01019,0.045076,...,0.239407,0.242023,0.086513,0.010136,0.030336,0.008045,0.121836,NaN,1.0,145.0
1,theophylline,DB00277,asthma,DOID:2841,8,more eff,0.000000,0.220995,0.00000,0.000000,...,0.000909,0.043457,0.000000,0.000000,0.000000,0.000000,0.201669,NaN,4.0,41.0
2,epinephrine,DB00668,asthma,DOID:2841,0.03,more eff,0.077547,0.179390,0.00000,0.000000,...,0.140446,0.058065,0.273700,0.000000,0.013580,0.000000,0.013580,NaN,2.0,19.0
3,orciprenaline,DB00816,asthma,DOID:2841,6,more eff,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,query drug and disease connected by genes not ...,3.0,0.0
4,salmeterol,DB00938,asthma,DOID:2841,5.5,none,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,query drug and disease not connected by genes ...,0.0,NaN
